# **Music Genre Classification**

### **Importing required libraries and packages**

In [1]:
!pip install python_speech_features
from python_speech_features import mfcc
import scipy.io.wavfile as wav
import numpy as np

from tempfile import TemporaryFile
import os
import pickle
import random 
import operator

import math

  Created wheel for python-speech-features: filename=python_speech_features-0.6-cp37-none-any.whl size=5887 sha256=9dfa35f8b5b7c535032393066852b8ded20d4ba449451a1062e52596a8a5feea
  Stored in directory: /root/.cache/pip/wheels/3c/42/7c/f60e9d1b40015cd69b213ad90f7c18a9264cd745b9888134be
Successfully built python-speech-features


**Defining a Function to find distance between neighbours.**


In [2]:
def distance(instance1 , instance2 , k ):
    distance =0 
    mm1 = instance1[0] 
    cm1 = instance1[1]
    mm2 = instance2[0]
    cm2 = instance2[1]
    distance = np.trace(np.dot(np.linalg.inv(cm2), cm1)) 
    distance+=(np.dot(np.dot((mm2-mm1).transpose() , np.linalg.inv(cm2)) , mm2-mm1 )) 
    distance+= np.log(np.linalg.det(cm2)) - np.log(np.linalg.det(cm1))
    distance-= k
    return distance

**Defining Function for finding Nearest Neighbours.**




In [3]:
def getNeighbors(trainingSet, instance, k):
    distances = []
    for x in range (len(trainingSet)):
        dist = distance(trainingSet[x], instance, k )+ distance(instance, trainingSet[x], k)
        distances.append((trainingSet[x][2], dist))
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for x in range(k):
        neighbors.append(distances[x][0])
    return neighbors

**Defining a Function for finding Class of the Neighbours.**

In [4]:
def nearestClass(neighbors):
    classVote = {}

    for x in range(len(neighbors)):
        response = neighbors[x]
        if response in classVote:
            classVote[response]+=1 
        else:
            classVote[response]=1

    sorter = sorted(classVote.items(), key = operator.itemgetter(1), reverse=True)
    return sorter[0][0]

**Defining a Function for finding out the accuracy of our model**

In [5]:
def getAccuracy(testSet, predictions):
    correct = 0 
    for x in range (len(testSet)):
        if testSet[x][-1]==predictions[x]:
            correct+=1
    return 1.0*correct/len(testSet)

**Extracting Features from the dataset**

In [6]:
directory = "/content/drive/MyDrive/genres"
f= open("my.dat" ,'wb')
i=0
for folder in os.listdir(directory):
    print(folder)
    i+=1
    if i==11 :
        break   
    for file in os.listdir(directory+'/'+folder):  
        (rate,sig) = wav.read(directory+'/'+folder+'/'+file)
        mfcc_feat = mfcc(sig,rate ,winlen=0.020, appendEnergy = False)
        covariance = np.cov(np.matrix.transpose(mfcc_feat))
        mean_matrix = mfcc_feat.mean(0)
        feature = (mean_matrix , covariance , i)
        pickle.dump(feature , f)

f.close()

disco
country
pop
blues
metal
classical
rock
reggae
jazz
hiphop


**Training the model and obtaining the Accuracy**

In [7]:
dataset = []
def loadDataset(filename , split , trSet , teSet):
    with open("my.dat" , 'rb') as f:
        while True:
            try:
                dataset.append(pickle.load(f))
            except EOFError:
                f.close()
                break  

    for x in range(len(dataset)):
        if random.random() <split :      
            trSet.append(dataset[x])
        else:
            teSet.append(dataset[x])  

trainingSet = []
testSet = []
loadDataset("my.dat" , 0.66, trainingSet, testSet)
leng = len(testSet)
predictions = []
for x in range (leng):
    predictions.append(nearestClass(getNeighbors(trainingSet ,testSet[x] , 5))) 

accuracy1 = getAccuracy(testSet , predictions)
print(accuracy1)


0.6918429003021148


**Testing the model with new music files.**

In [10]:
from collections import defaultdict
results=defaultdict(int)

i=1
for folder in os.listdir("/content/drive/MyDrive/genres"):
    results[i]=folder
    i+=1
print(results)

for file in os.listdir("/content/drive/MyDrive/test"):
  (rate,sig)=wav.read("/content/drive/MyDrive/test"+'/'+file)
  mfcc_feat=mfcc(sig,rate,winlen=0.020,appendEnergy=False)
  covariance = np.cov(np.matrix.transpose(mfcc_feat))
  mean_matrix = mfcc_feat.mean(0)
  feature=(mean_matrix,covariance,0)

  pred=nearestClass(getNeighbors(dataset ,feature , 5))

  print(file,':',results[pred])

defaultdict(<class 'int'>, {1: 'disco', 2: 'country', 3: 'pop', 4: 'blues', 5: 'metal', 6: 'classical', 7: 'rock', 8: 'reggae', 9: 'jazz', 10: 'hiphop'})
test-hiphop.wav : hiphop
test-disco.wav : disco
test-country.wav : country
test-classical.wav : classical
test-blues.wav : blues
test-rock.wav : reggae
test-reggae.wav : reggae
test-pop.wav : disco
test-metal.wav : metal
test-jazz.wav : jazz
